# Test notebook for Piecewise Linear Utility Androids

In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `c:\Users\gggzy\Desktop\Z\mkt\ExchangeMarket.jl\scripts`


In [14]:
using Revise
using Random, SparseArrays, LinearAlgebra
using JuMP, MosekTools
using Plots, LaTeXStrings, Printf
import MathOptInterface as MOI

using ExchangeMarket

include("../tools.jl")
include("../plots.jl")
switch_to_pdf(; bool_use_html=true)

include("./util.jl")
include("./master.jl")
include("./pricing.jl")

update_pwl_optimizer_from_market!

## Setup Piecewise Linear Market

In [15]:
m = 3  # number of ground truth agents
n = 10  # number of goods
K = 12  # number of revealed preference observations

# -----------------------------------------------------------------------
# Create ground truth market with piecewise linear agents
# -----------------------------------------------------------------------
L_max = 4  # maximum number of segments per good
u_A_gt = zeros(Float64, n, L_max, m)
q_A_gt = zeros(Float64, n, L_max, m)

Random.seed!(42)
for i in 1:m
    for j in 1:n
        L_j = rand(2:L_max)  # random number of segments for good j
        a_prev = 10.0  # starting marginal utility
        for ℓ in 1:L_j
            # Sample decreasing marginal utilities
            a_jℓ = rand() * (a_prev / 2) + (a_prev / 2)
            u_A_gt[j, ℓ, i] = a_jℓ
            # Sample capacities
            q_A_gt[j, ℓ, i] = rand() * (10.0 - 0.1) + 0.1
            a_prev = a_jℓ
        end
    end
end

# Create FisherMarket with piecewise linear utilities
f0 = FisherMarket(m, n; u_A=u_A_gt, q_A=q_A_gt, scale=1.0)
linconstr = LinearConstr(1, n, ones(1, n), [1.0])

# -----------------------------------------------------------------------
# Setup algorithm for computing demands
# -----------------------------------------------------------------------
f1 = copy(f0)
p₀ = ones(n) ./ n
x₀ = ones(n, m) ./ m
f1.x .= x₀
# Convert u_A and q_A to pwl_params format for PiecewiseLPResponse
pwl_params = convert_uA_qA_to_pwl_params(u_A_gt, q_A_gt)
println(typeof(pwl_params))
pwl_optimizer = PiecewiseLPResponse(pwl_params)

# Use MirrorDescent for piecewise linear 
alg = MirrorDec(
    n, m, p₀;
    optimizer=pwl_optimizer,
    option_grad=:dual,
    option_step=:eg,
    option_stepsize=:cc13,
    tol=1e-6,
    maxiter=1000
)

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0000 seconds
FisherMarket initialized in 0.0010 seconds
FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0000 seconds
FisherMarket initialized in 0.0000 seconds
Dict{Int64, Vector{Vector{Tuple{Float64, Float64}}}}


MirrorDec{Float64}(10, 3, [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.786400165183776, 0.8925975682484205, 0.45201517659689416, 0.2068733032049488, 0.6721739949391414, 0.630176047635695, 0.24998573762663634, 0.6379494743238538, 0.04440825613482957, 0.4058890219614756], -1.0e6, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 2.5e-323, 1.276646706614e-311, [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0], 3.0e-323, 1.77004526476e9, 1.0e-322, 4.0e-323, 1.276646706614e-311, 0, 0, 1000, 100.0, 1.0e-6, :MirrorDec, ExchangeMarket.ResponseOptimizer(ExchangeMarket.var"#__piecewise_response#110"{ExchangeMarket.var"#__piecewise_response#105#111"{Dict{Int64, Vector{Vector{Tuple{Float64, Float64}}}}}}(ExchangeMarket.var"#__piecewise_response#105#111"{Dict{Int64, Vector{Vector{Tuple{Float64, Float64}}}}}(Dict(2 => [[(5.330344955327541, 2.025655484514394), (5.23389325789074, 3.6581030108423405), (4.6562983884097, 7.1415980

## Generate Revealed Preferences

In [16]:
# Generate revealed preferences from the ground truth market
Ξ = produce_revealed_preferences(alg, f1, K; seed=42)

println("Generated ", length(Ξ), " revealed preference observations")
println("First observation: p = ", Ξ[1][1], "\n  g = ", Ξ[1][2])

Computing demand for observation 1 at prices: [0.06712779654090693, 0.08673837419857172, 0.0783854508486898, 0.06618281351094174, 0.11393250590798745, 0.10810670523914472, 0.09587600391262513, 0.15044865550529463, 0.13975130600358215, 0.09345038833225555]
Computing demand for observation 2 at prices: [0.0631127855918965, 0.09512666932135919, 0.1893884563288241, 0.050329934926855675, 0.15827857041487253, 0.09885903628706368, 0.10074186897540965, 0.08627713556098958, 0.09982645451589879, 0.05805908807683045]
Computing demand for observation 3 at prices: [0.12627762461564074, 0.08170623683264273, 0.11674698833770944, 0.07152184212464718, 0.04279456415619958, 0.1570731445702084, 0.05516702194181713, 0.13539069941094237, 0.0697265042449877, 0.14359537376520456]
Computing demand for observation 4 at prices: [0.08237963375514741, 0.12022582534388461, 0.11561714928107493, 0.12782251754944593, 0.12016882844798792, 0.141248098455155, 0.10074554599418517, 0.04627790645995315, 0.09562326080309651,

## Test Master Problem with Demand Vectors

In [17]:
# Compute demand matrix from ground truth market
x_gt = compute_demand_from_market(f1, Ξ, alg)

# Solve master problem using demand vectors
λ, s, model_primal = solve_master_problem_piecewise(Ξ, x_gt; verbose=true)
u, μ, model_dual = solve_dual_problem_piecewise(Ξ, x_gt; verbose=true)

# Validate strong duality
println("=== Strong Duality Validation ===")
println("Primal objective (Q):     ", objective_value(model_primal))
println("Dual objective (Q_*):     ", objective_value(model_dual))
println("Gap:                      ", abs(objective_value(model_primal) - objective_value(model_dual)))
println()
println("Primal solution λ - ground truth:   ", abs.(λ - f1.w) |> maximum)
println("Dual solution μ:          ", μ)
println("Dual solution u (first 5): ", u[1:5, :])

Set parameter OutputFlag to value 1
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i9-14900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 361 rows, 135 columns and 675 nonzeros
Model fingerprint: 0x3fee7be0
Coefficient statistics:
  Matrix range     [1e-01, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 2e+01]
Presolve removed 296 rows and 120 columns
Presolve time: 0.00s
Presolved: 65 rows, 15 columns, 234 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4454022e+00   5.133710e+01   0.000000e+00      0s
      21    5.7292563e+01   0.000000e+00   0.000000e+00      0s

Solved in 21 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.729256344e+01

User-callback calls 98, time in user-callbac

## Column Generation for Piecewise Linear Agents

In [18]:
# Column generation: iteratively add new piecewise linear androids to fit revealed preferences
# -----------------------------------------------------------------------
# Parameters
# -----------------------------------------------------------------------
max_iters = 50
tol = 1e-6  # tolerance for reduced cost

# -----------------------------------------------------------------------
# Initialize surrogate market with one random piecewise linear agent
# -----------------------------------------------------------------------
u_A_init, q_A_init = generate_random_piecewise_params(n; L_max=4, a_max=10.0, q_min=0.1, q_max=10.0)
fa = FisherMarket(1, n; u_A=u_A_init, q_A=q_A_init, scale=1.0)

# Compute initial demand matrix
x_ref = Ref(compute_demand_from_market(fa, Ξ, alg))

# Track history
history = Dict(
    :primal_obj => Float64[],
    :dual_obj => Float64[],
    :reduced_cost => Float64[],
    :num_agents => Int[]
)

println("=== Column Generation for Piecewise Linear Android Fitting ===\n")

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0170 seconds
FisherMarket initialized in 0.0170 seconds
=== Column Generation for Piecewise Linear Android Fitting ===



In [19]:
for iter in 1:max_iters
    println("--- Iteration $iter ($(fa.m) agents) ---")

    # Solve master problem (primal and dual) using demand vectors
    λ, s, model_primal = solve_master_problem_piecewise(Ξ, x_ref[]; verbose=false)
    u, μ, model_dual = solve_dual_problem_piecewise(Ξ, x_ref[]; verbose=false)

    primal_obj = objective_value(model_primal)
    dual_obj = objective_value(model_dual)

    push!(history[:primal_obj], primal_obj)
    push!(history[:dual_obj], dual_obj)
    push!(history[:num_agents], fa.m)

    println("  Primal obj: $(round(primal_obj, digits=6))")
    println("  Dual obj:   $(round(dual_obj, digits=6))")
    println("  Weights λ:  $(round.(λ, digits=4))")

    # Solve pricing subproblem to find best new piecewise linear agent
    u_A_new, q_A_new, x_new, pricing_obj = solve_pricing_piecewise(Ξ, u; M=20, verbose=false)

    # Compute reduced cost
    rc = reduced_cost_piecewise(x_new, u, μ)
    push!(history[:reduced_cost], rc)

    println("  Reduced cost: $(round(rc, digits=6))")

    # Check termination: if reduced cost <= 0, no improving agent exists
    if rc <= tol
        println("\n✓ Converged! No improving agent found (reduced cost=$rc ≤ $tol)")
        break
    end

    # Update existing agents' budgets with optimal weights from master
    fa.w .= λ

    # Add new agent's demand vectors to the demand matrix
    add_to_demand!(x_ref, x_new)

    # Add new piecewise linear agent to surrogate market
    w_new = 0.0  # placeholder, will be updated in next iteration
    add_piecewise_to_market!(fa, u_A_new, q_A_new, w_new)

    println("  → Added agent $(fa.m)")
    println()

    if iter == max_iters
        println("\n⚠ Maximum iterations reached")
    end
end

# Final solve to get optimal weights for all agents
λ_final, _, _ = solve_master_problem_piecewise(Ξ, x_ref[]; verbose=false)
fa.w .= λ_final

println("\n=== Final Results ===")
println("Number of fitted agents: $(fa.m)")
println("Number of nonzero agents: $(sum(λ_final .> 1e-6))")
println("Ground truth agents:     $(f1.m)")
println("Final primal objective:  $(round(history[:primal_obj][end], digits=6))")
println("Final weights: $(round.(λ_final, digits=4))")

--- Iteration 1 (1 agents) ---
  Primal obj: 110.05693
  Dual obj:   105.026907
  Weights λ:  [1.0]
  Reduced cost: 51.415626
  → Added agent 2

--- Iteration 2 (2 agents) ---
  Primal obj: 69.950201
  Dual obj:   64.216643
  Weights λ:  [0.179, 0.821]
  Reduced cost: 39.518657
  → Added agent 3

--- Iteration 3 (3 agents) ---
  Primal obj: 62.320874
  Dual obj:   59.251626
  Weights λ:  [0.0, 0.764, 0.236]
  Reduced cost: 7.866475
  → Added agent 4

--- Iteration 4 (4 agents) ---
  Primal obj: 62.320874
  Dual obj:   58.068951
  Weights λ:  [0.0, 0.764, 0.236, 0.0]
  Reduced cost: 6.318327
  → Added agent 5

--- Iteration 5 (5 agents) ---
  Primal obj: 60.689048
  Dual obj:   57.797634
  Weights λ:  [0.0, 0.6386, 0.2112, 0.0, 0.1501]
  Reduced cost: 23.185211
  → Added agent 6

--- Iteration 6 (6 agents) ---
  Primal obj: 54.663808
  Dual obj:   52.413428
  Weights λ:  [0.0, 0.4163, 0.0, 0.1703, 0.202, 0.2113]
  Reduced cost: 2.763013
  → Added agent 7

--- Iteration 7 (7 agents) ---


## Validate Fitted Market

In [20]:
# Test if fitted market reproduces observed demands
p1, g1 = Ξ[1]
alg.p .= p1
play!(alg, fa)
ga = sum(fa.x, dims=2)[:]

println("Observed aggregate demand: ", g1)
println("Fitted aggregate demand:   ", ga)
println("Difference:                ", abs.(ga - g1))
println("Max difference:            ", maximum(abs.(ga - g1)))

Observed aggregate demand: [9.679028777579303, 0.0, 0.0, 5.292433290814463, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Fitted aggregate demand:   [10.211360755069656, 0.0, 0.0, 4.752500477724201, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Difference:                [0.5323319774903528, 0.0, 0.0, 0.5399328130902621, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Max difference:            0.5399328130902621


## Plot Convergence History

In [21]:
using Plots

p1 = plot(
    history[:primal_obj],
    label="Primal Objective",
    xlabel="Iteration",
    ylabel="Objective Value",
    title="Column Generation Convergence"
)
plot!(p1, history[:dual_obj], label="Dual Objective")
display(p1)

p2 = plot(
    history[:reduced_cost],
    label="Reduced Cost",
    xlabel="Iteration",
    ylabel="Reduced Cost",
    title="Pricing Subproblem"
)
display(p2)

p3 = plot(
    history[:num_agents],
    label="Number of Agents",
    xlabel="Iteration",
    ylabel="Agents",
    title="Agent Growth"
)
display(p3)